<a href="https://colab.research.google.com/github/PhoenixCC0722/Journey_to_become_DataScientist/blob/main/3_1_5_Kopie_quality_assessment_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quality Assessment
Now it's your turn. Follow [the steps on the platform](https://learn.wbscodingschool.com/courses/data-science/lessons/%f0%9f%93%9d-5-data-quality/) and use what you've learnt to see how reliable the data is.

In [13]:
import pandas as pd

Load our cleaned DataFrames

In [14]:
# orders_cl.csv
url = "https://drive.google.com/file/d/1Tla62vfu__kCqvgypZyVt2S9VuC016yH/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orders_cl = pd.read_csv(path)

# orderlines_cl.csv
url = "https://drive.google.com/file/d/1OhtkQS2fwOYdzfd-qPh7im35iLc-L9TA/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
orderlines_cl = pd.read_csv(path)

# products_cl.csv
url = "https://drive.google.com/file/d/1s7Lai4NSlsYjGEPg1QSOUJobNYVsZBOJ/view?usp=sharing"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
products_cl = pd.read_csv(path)

## 1.&nbsp; Define Pandas display format

In [15]:
pd.set_option('display.float_format', lambda x: '%.2f' % x) # format floating-point numbers to two decimal places
pd.set_option('display.max_rows',1000)

## 2.&nbsp; Exclude unwanted orders

In [ ]:
"""
# 2nd, make sure that orders in the dataset are in both orders and orderlines
# orders –> Keep only orders with the states that you want to work with.
or_df = orders_cl.copy()
#print(or_df.state.unique()) # ['Cancelled' 'Completed' 'Pending' 'Shopping Basket' 'Place Order']
or_df.state.value_counts()
filt_or_df = or_df.loc[or_df.state == 'Completed',:]
#print(filt_or_df.state.unique())
# orderlines –> Keep only orders present in orders.
# orders –> Keep only orders present in orderlines.
ol_df = orderlines_cl.copy()
print(ol_df.columns, filt_or_df.columns, sep='\n')
ol_df = ol_df.rename(columns={'id_order':'order_id'})
merge_df = pd.merge(ol_df, filt_or_df, on='order_id', how='inner')
merge_df.head(20)
"""

Step 1. choose only the order with the state are completed, first we have to Let's create a copy of the orders_cl DataFrame, to make sure any of our changes won't affect the original DataFrame.

In [17]:
or_df = orders_cl.copy()
#print(or_df.state.unique()) # ['Cancelled' 'Completed' 'Pending' 'Shopping Basket' 'Place Order']
or_df.state.value_counts()

Shopping Basket    117809
Completed           46605
Place Order         40883
Pending             14374
Cancelled            7233
Name: state, dtype: int64

Step 2. Seems like a hugh amount of the orders are not completed, we are only interested in the completed orders

In [18]:
completed_or_df = or_df.loc[or_df.state == 'Completed',:]
completed_or_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46605 entries, 1 to 226614
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      46605 non-null  int64  
 1   created_date  46605 non-null  object 
 2   total_paid    46605 non-null  float64
 3   state         46605 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 1.8+ MB


Step 2. Alternative filtering method

In [ ]:
# rows_to_drop = ['Cancelled','Pending','Shopping Basket','Place Order']
# completed_or_df = or_df.loc[~or_df['state'].isin(rows_to_drop)]

# mask = or_df['state'].isin(['Cancelled','Pending','Shopping Basket','Place Order'])
# completed_or_df = or_df.loc[or_df[mask].index[:]]

Step 2-1. is there any duplicates proudcts in the order dataframe

In [19]:
len(list(completed_or_df.order_id.unique())) #46605, no duplicates

46605

Step 3. merge the completed_or_df with the ol_df together with innerjoin, then the unwanted order will not included

In [21]:
ol_df = orderlines_cl.copy()
merge_completed_or_ol_df = pd.merge(completed_or_df, ol_df, how='inner', left_on='order_id', right_on='id_order')
len(list(merge_completed_or_ol_df.order_id.unique()))

43064

## 3.&nbsp; Exclude orders with unknown products


3.1 first have to check out how many sku categories are there and if there are any duplicated values

In [32]:
pd_df = products_cl.copy()
pd_df.sku.nunique() #9992
pd_df.sku.duplicated().sum() # no duplicated values

0

3.2 wenn die sku not in the products Dataframe, then we delete the whole row in both the orders and orderline tables

In [38]:
print(merge_completed_or_ol_df.shape[0])

56714


In [40]:
merge_df_exclude = merge_completed_or_ol_df.loc[merge_completed_or_ol_df.sku.isin(pd_df['sku']),:]
#len(list(merge_df_exclude.sku.unique()))
#merge_df_exclude.head()
len(list(merge_df_exclude.order_id.unique()))
#print(merge_df_exclude.shape[0])

42259

# 4.  Explore the revenue from different tables

#### Step 1:
Create the `unit_price_total` as `orderlines.unit_price` * `orderlines.product_quantity`

In [42]:
merge_df_exclude.head()

,order_id,created_date,total_paid,state,id,id_order,product_id,product_quantity,sku,unit_price,date
0,241423,2017-11-06 13:10:02,136.15,Completed,1398738,241423,0,1,LAC0212,129.16,2017-11-06 12:47:20
1,242832,2017-12-31 17:40:03,15.76,Completed,1529178,242832,0,1,PAR0074,10.77,2017-12-31 17:26:40
2,243330,2017-02-16 10:59:38,84.98,Completed,1181923,243330,0,1,OWC0074,77.99,2017-02-15 17:07:44
3,245275,2017-06-28 11:35:37,149.00,Completed,1276706,245275,0,1,TAD0007,149.00,2017-06-28 11:12:30
4,245595,2017-01-21 12:52:47,112.97,Completed,1154394,245595,0,2,PAC1561,52.99,2017-01-21 12:49:00


In [44]:
# your code here
merge_df_exclude['unit_price_total'] = merge_df_exclude['unit_price'] * merge_df_exclude['product_quantity']
merge_df_exclude.head()

<ipython-input-44-231261340d82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_df_exclude['unit_price_total'] = merge_df_exclude['unit_price'] * merge_df_exclude['product_quantity']


,order_id,created_date,total_paid,state,id,id_order,product_id,product_quantity,sku,unit_price,date,unit_price_total
0,241423,2017-11-06 13:10:02,136.15,Completed,1398738,241423,0,1,LAC0212,129.16,2017-11-06 12:47:20,129.16
1,242832,2017-12-31 17:40:03,15.76,Completed,1529178,242832,0,1,PAR0074,10.77,2017-12-31 17:26:40,10.77
2,243330,2017-02-16 10:59:38,84.98,Completed,1181923,243330,0,1,OWC0074,77.99,2017-02-15 17:07:44,77.99
3,245275,2017-06-28 11:35:37,149.00,Completed,1276706,245275,0,1,TAD0007,149.00,2017-06-28 11:12:30,149.00
4,245595,2017-01-21 12:52:47,112.97,Completed,1154394,245595,0,2,PAC1561,52.99,2017-01-21 12:49:00,105.98


#### Step 2:
Group by `id_order`, summarising by the sum of `unit_price_total`

In [56]:
# id_order already renamed as order_id
gp_ol_df = merge_df_exclude.groupby('order_id')['unit_price_total'].sum().reset_index()
gp_ol_df.head()
gp_ol_df.isna().sum()

order_id            0
unit_price_total    0
dtype: int64

### What is the average difference between `total_paid` and `unit_price_total`?

In [57]:
#gp_ol_df = gp_ol_df.to_frame(name='unit_price_total')
mg_or_gp_ol = pd.merge(completed_or_df, gp_ol_df, how='left', on='order_id')
#mg_or_gp_ol.head()
mg_or_gp_ol['difference'] = mg_or_gp_ol.total_paid - mg_or_gp_ol.unit_price_total
print(mg_or_gp_ol['difference'].mean()) # 5.934
#mg_or_gp_ol.head(25)

5.934310797699899


### What is the distribution of these differences?

In [54]:
# distribution
print(mg_or_gp_ol['difference'].value_counts().head(10), mg_or_gp_ol['difference'].value_counts(normalize=True).head(10), sep='\n')
#mg_or_gp_ol['difference'].nunique()

0.00    10484
6.99     4559
4.99     3419
4.99     2523
4.99     2352
6.99     1890
3.99     1882
3.99     1620
3.99     1463
6.99     1305
Name: difference, dtype: int64
0.00   0.25
6.99   0.11
4.99   0.08
4.99   0.06
4.99   0.06
6.99   0.04
3.99   0.04
3.99   0.04
3.99   0.03
6.99   0.03
Name: difference, dtype: float64


### Can all the differences be explained by shipping costs? If not, what are other plausible explanations?

In [ ]:
# your explanation here
mg_or_gp_ol.loc[(mg_or_gp_ol['difference'] >= 7.0),:] #2861 rows × 6 columns

,order_id,created_date,total_paid,state,unit_price_total,difference
8,246405,2017-11-24 10:01:27,407.96,Completed,275.75,132.21
13,251688,2017-11-24 23:57:07,505.76,Completed,485.77,19.99
14,251969,2017-11-21 21:04:12,610.07,Completed,590.08,19.99
17,253306,2017-10-04 14:14:45,497.68,Completed,477.68,20.00
22,254537,2017-05-23 19:58:30,102.97,Pending,32.99,69.98
...,...,...,...,...,...,...
101652,527020,2018-03-14 12:55:02,506.98,Pending,499.98,7.00
101712,527125,2018-03-14 12:00:55,73.70,Place Order,53.71,19.99
101738,527167,2018-03-14 12:18:54,361.56,Pending,354.56,7.00
101810,527294,2018-03-14 13:08:14,182.96,Pending,175.96,7.00


### If there are differences that you can’t explain: what should you do with these orders?

In [ ]:
# your code here

## 5.&nbsp; Become confident about your dataset

Let's hear your thoughts about what you discovered.

In [ ]:
# your thoughts & code here

Do not forget to download your quality controlled DataFrames

In [ ]:
from google.colab import files

#orders_quality_controlled.to_csv("orders_qu.csv", index=False)
#files.download("orders_qu.csv")

#orderlines_quality_controlled.to_csv("orderlines_qu.csv", index=False)
#files.download("orderlines_qu.csv")